# PROCESADO DEL CSV ORIGINAL DE KAGGLE PARA SACAR EL DATAFRAME TRAIN

In [463]:
import pandas as pd
import numpy as np

## Cargamos los datos de entrada del archivo .csv Train

In [464]:
# Cargamos los datos de entrada del archivo Train.csv
train = pd.read_csv('../data/Raw/train.csv', sep=',')
df_train=pd.DataFrame(train)
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [465]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [466]:
# Hay 2 opciones en la columna 'HomePlanet'. Como son campos de tipo object, osea, string, los paso a 0 ó 1 con el metodo getdummies y los guardo en un dataframe temporal.
planetas=pd.get_dummies(df_train['HomePlanet'])
planetas

,Earth,Europa,Mars
0,0,1,0
1,1,0,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
8688,0,1,0
8689,1,0,0
8690,1,0,0
8691,0,1,0


In [467]:
df_train=pd.concat([df_train, planetas], axis=1)

In [468]:
df_train=df_train.fillna(df_train.median())
df_train['Cabin']= df_train['Cabin'].astype('string')

c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [469]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep         0
Cabin           199
Destination     182
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Earth             0
Europa            0
Mars              0
dtype: int64

In [470]:
# Separo la columna cabin en 3 para crear mas tarde 3 nuevas columnas con sus correspondientes valores sacados de esta
df_train['Cabin_Side'] = df_train['Cabin'].str.split("/").str[2]
df_train['Cabin_num'] = df_train['Cabin'].str.split("/").str[1]
df_train['Cabin_Deck'] = df_train['Cabin'].str.split("/").str[0]

In [471]:
# Separo la columna 'Cabin_Side' en 2, porq tiene 2 valores,P y S y las guardo 
cabinas = pd.get_dummies(df_train['Cabin_Side'], prefix='Cabin_Side')
df_train=pd.concat([df_train, cabinas], axis=1)

In [472]:
# Convertimos la columna Cabin_deck a numerica con getdummies y la guardo en columnas individuales separadas
cabinas = pd.get_dummies(df_train['Cabin_Deck'], prefix='Cabin_Deck')
df_train=pd.concat([df_train, cabinas], axis=1)

In [473]:
# El campo 'Cabin_num', tiene valores . Lo relleno provisionalmente cn nueves y despues con NAN que son valores nulos pero de typo Float :)
df_train['Cabin_num']=df_train['Cabin_num'].fillna('-99999999')
df_train['Cabin_num'].replace({"-99999999": np.nan})
df_train['Cabin_num'].value_counts

<bound method IndexOpsMixin.value_counts of 0          0
1          0
2          0
3          0
4          1
        ... 
8688      98
8689    1499
8690    1500
8691     608
8692     608
Name: Cabin_num, Length: 8693, dtype: object>

In [474]:
#Borramos las columnas que he ido trasformando y que ya no me sirven
df_train.drop('Cabin_Side', axis=1, inplace=True)
df_train.drop('HomePlanet', axis=1, inplace=True)
df_train.drop('Destination', axis=1, inplace=True)
df_train.drop('Name', axis=1, inplace=True)
df_train.drop('Cabin',axis=1, inplace=True)
df_train.drop('Cabin_Deck',axis=1, inplace=True)

#Transformamos la columna VIP, q esta cm string a tipo int
df_train['VIP'] = df_train['VIP'].replace({False: 0, True: 1}).astype(int)
df_train['Cabin_num']=df_train['Cabin_num'].astype(int)

#Transformamos la columna CryoSleep, q esta cm string a tipo int
df_train['CryoSleep'] = df_train['CryoSleep'].replace({False: 0, True: 1}).astype(int)
df_train['Cabin_num']=df_train['Cabin_num'].astype(int)

df_train=df_train.fillna(df_train.median())
df_train = df_train.dropna()


In [475]:
# Ahora si...ataframe limpio de valores nulos y sin tipos de campos String o Object oncompatibles a la hora de crear el modelo y entrenarlo
df_train.isnull().sum()

PassengerId     0
CryoSleep       0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Earth           0
Europa          0
Mars            0
Cabin_num       0
Cabin_Side_P    0
Cabin_Side_S    0
Cabin_Deck_A    0
Cabin_Deck_B    0
Cabin_Deck_C    0
Cabin_Deck_D    0
Cabin_Deck_E    0
Cabin_Deck_F    0
Cabin_Deck_G    0
Cabin_Deck_T    0
dtype: int64

In [476]:
df_train

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,Cabin_Side_P,Cabin_Side_S,Cabin_Deck_A,Cabin_Deck_B,Cabin_Deck_C,Cabin_Deck_D,Cabin_Deck_E,Cabin_Deck_F,Cabin_Deck_G,Cabin_Deck_T
0,0001_01,0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,...,1,0,0,1,0,0,0,0,0,0
1,0002_01,0,24.0,0,109.0,9.0,25.0,549.0,44.0,True,...,0,1,0,0,0,0,0,1,0,0
2,0003_01,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,...,0,1,1,0,0,0,0,0,0,0
3,0003_02,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,...,0,1,1,0,0,0,0,0,0,0
4,0004_01,0,16.0,0,303.0,70.0,151.0,565.0,2.0,True,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,...,1,0,1,0,0,0,0,0,0,0
8689,9278_01,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,...,0,1,0,0,0,0,0,0,1,0
8690,9279_01,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,...,0,1,0,0,0,0,0,0,1,0
8691,9280_01,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,...,0,1,0,0,0,0,1,0,0,0


In [477]:
# Ahora si, todas las features que voy a entrenar son de typo numerico o bool y sin nulos
df_train.dtypes

PassengerId      object
CryoSleep         int32
Age             float64
VIP               int32
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
Earth             uint8
Europa            uint8
Mars              uint8
Cabin_num         int32
Cabin_Side_P      uint8
Cabin_Side_S      uint8
Cabin_Deck_A      uint8
Cabin_Deck_B      uint8
Cabin_Deck_C      uint8
Cabin_Deck_D      uint8
Cabin_Deck_E      uint8
Cabin_Deck_F      uint8
Cabin_Deck_G      uint8
Cabin_Deck_T      uint8
dtype: object

In [478]:
df_train.to_csv('../data/Processed/train_processed.csv', sep=',', index=False)